In [5]:
from util import parse_graph_from_edgelist_file, visualize_fvs
from solver import FeedbackVertexSetSolverSAT
from greedy import greedy_fvs
import networkx as nx
import time
import threading

### Load the test graph, draw it using a "nice" layout.

In [6]:
"""
G = parse_graph_from_edgelist_file("instances/graph1.edges")
# G = nx.generators.fast_gnp_random_graph(25, 0.3)
nx.draw_networkx(G, pos=nx.layout.kamada_kawai_layout(G))
"""

'\nG = parse_graph_from_edgelist_file("instances/graph1.edges")\n# G = nx.generators.fast_gnp_random_graph(25, 0.3)\nnx.draw_networkx(G, pos=nx.layout.kamada_kawai_layout(G))\n'

## Find a greedy solution

In [7]:
"""
greedy_solution = greedy_fvs(G)
assert nx.is_forest(G.subgraph(set(G.nodes()) - greedy_solution))

print(f"The greedy solution {greedy_solution} has size {len(greedy_solution)}!")
visualize_fvs(G, greedy_solution)
"""

'\ngreedy_solution = greedy_fvs(G)\nassert nx.is_forest(G.subgraph(set(G.nodes()) - greedy_solution))\n\nprint(f"The greedy solution {greedy_solution} has size {len(greedy_solution)}!")\nvisualize_fvs(G, greedy_solution)\n'

## Use the exact solver using Cardinality SAT to find the optimal solution

In [8]:
"""
fbv_solver = FeedbackVertexSetSolverSAT(G)
optimal_solution = fbv_solver.find_optimum()
print(optimal_solution)
print(f"The optimal solution is {round(100.0 -  100.0 * len(optimal_solution) / len(greedy_solution), ndigits=2)}% smaller than the heuristic solution!")
visualize_fvs(G, feedback_vertex_set=optimal_solution)

assert nx.is_forest(G.subgraph(set(G.nodes()) - optimal_solution))
"""

# Define a function to run the find_optimum method in a separate thread
def run_solver(fbv_solver):
    global optimal_solution
    optimal_solution = fbv_solver.find_optimum()

# Start with a small number of nodes
num_nodes = 30

# Increment the number of nodes by 10 for each iteration
increment = 1

# Set the time limit to 5 minutes
time_limit = 300  # seconds

# Keep track of the largest instance that can be solved within the time limit
largest_size = num_nodes
start_time = time.time()
while time.time() - start_time < 300:
    # Generate a random graph with the specified number of nodes and edge probability
    G = nx.generators.fast_gnp_random_graph(num_nodes, 0.3)

    # Measure the time taken to solve the instance
    fbv_solver = FeedbackVertexSetSolverSAT(G)

    # Start a separate thread to run the find_optimum method
    t = threading.Thread(target=run_solver, args=(fbv_solver,))
    t.start()

    # Wait for the specified time limit
    t.join(timeout=time_limit)

    # If the separate thread is still running, stop it and return the largest instance that could be solved
    if t.is_alive():
        t.stop()
        print(f"Largest instance that can be solved within {time_limit} seconds: {largest_size} nodes")
        break
    elapsed_time = time.time() - start_time

    # If the time taken is less than the time limit, increase the number of nodes and try again
    if elapsed_time < time_limit:
        largest_size = num_nodes
        num_nodes += increment
    else:
        # If the time limit is reached, return the largest instance that can be solved
        print(f"Largest instance that can be solved within {time_limit} seconds: {largest_size} nodes")
        break

Number of cycles in cycle basis: 101
A greedy solution of size 16 was found!
k = 8: UNSAT
k = 12: UNSAT
k = 14: UNSAT
k = 15: SAT
A solution of size 15 was found by the solver!
The solution of size 15 was proven to be optimal!
Number of cycles in cycle basis: 115
A greedy solution of size 16 was found!
k = 8: UNSAT
k = 12: UNSAT
k = 14: SAT
A solution of size 14 was found by the solver!
k = 13: UNSAT
The solution of size 14 was proven to be optimal!
Number of cycles in cycle basis: 121
A greedy solution of size 19 was found!
k = 9: UNSAT
k = 14: UNSAT
k = 17: SAT
A solution of size 17 was found by the solver!
k = 16: UNSAT
The solution of size 17 was proven to be optimal!
Number of cycles in cycle basis: 146
A greedy solution of size 21 was found!
k = 10: UNSAT
k = 16: UNSAT
k = 19: SAT
A solution of size 19 was found by the solver!
k = 18: SAT
A solution of size 18 was found by the solver!
k = 17: UNSAT
The solution of size 18 was proven to be optimal!
Number of cycles in cycle basis: